In [22]:
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
im = Image.open('chad/1985.jpg')
im.show()

In [6]:
import os
import numpy

In [8]:
import os
sat_images = []

for filename in os.listdir('chad/'):
    if filename.endswith(".jpg"):
        sat_images.append(Image.open('chad/' + filename))

In [11]:
imarray = numpy.array(im)

In [25]:


for row in imarray:
    for pixel in row:
        

array([[[155, 146, 115],
        [144, 137, 108],
        [142, 135, 107],
        ...,
        [ 93,  94,  88],
        [ 96,  97,  92],
        [ 88,  90,  87]],

       [[155, 148, 119],
        [146, 139, 110],
        [145, 138, 110],
        ...,
        [ 86,  88,  83],
        [ 93,  95,  90],
        [ 91,  93,  88]],

       [[157, 150, 121],
        [148, 141, 113],
        [147, 140, 112],
        ...,
        [ 84,  89,  83],
        [ 90,  95,  89],
        [ 90,  95,  89]],

       ...,

       [[109, 105,  93],
        [109, 105,  93],
        [109, 105,  93],
        ...,
        [180, 164, 149],
        [178, 162, 147],
        [176, 160, 145]],

       [[111, 107,  95],
        [113, 109,  97],
        [114, 110,  98],
        ...,
        [173, 157, 142],
        [171, 155, 140],
        [171, 155, 140]],

       [[113, 109,  97],
        [114, 110,  98],
        [114, 110,  98],
        ...,
        [161, 145, 130],
        [156, 140, 125],
        [160, 144, 129]]